<a href="https://colab.research.google.com/github/premdub/AutoML/blob/main/location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install tpot mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import pandas as pd
location = pd.read_csv('/content/combined_df3.csv')
location

,Unnamed: 0,id,practice_id_,appointment_date_time,patient_zipcode,distance,appointment_yosi_noshow1,practice_id_update,practice_name,practice_zipcode
0,0,1165206,506,4/14/2021 12:14,19127.0,12.380316,N,506,Concorde Medical Group PLLC,10016
1,1,1165206,506,4/14/2021 12:14,19127.0,12.380316,N,506,Cornell Physicians - ENT,10021
2,2,1165206,506,4/14/2021 12:14,19127.0,12.380316,N,506,Weill Cornell Brain And Spine Center,11355
3,3,1165206,506,4/14/2021 12:14,19127.0,12.380316,N,506,Medical Associates Of Marlboro,8857
4,4,1165206,506,4/14/2021 12:14,19127.0,12.380316,N,506,vybe urgent care Roxborough,19128
...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,224565,501,9/30/2019 8:20,77084.0,19.588529,Y,501,Shift Integrative Medicine,10003
1048571,1048571,224565,501,9/30/2019 8:20,77084.0,19.588529,Y,501,Manhattan Pediatrics,10021
1048572,1048572,224565,501,9/30/2019 8:20,77084.0,19.588529,Y,501,"Medical Imaging of Manhattan, LLC",10022
1048573,1048573,224565,501,9/30/2019 8:20,77084.0,19.588529,Y,501,NYC Back Chiropractic,10128


2...Whether distance between provider location and patient location predict the SHOW or NO SHOW

In [33]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML


In [34]:
location.columns

Index(['Unnamed: 0', 'id', 'practice_id_', 'appointment_date_time',
       'patient_zipcode', 'distance', 'appointment_yosi_noshow1',
       'practice_id_update', 'practice_name', 'practice_zipcode'],
      dtype='object')

Potential variables of interest

1.  appointment_yosi_noshow1(categorical)
2.  distance(continuous)




In [17]:
location['appointment_yosi_noshow1'].describe()

count     1048575
unique          2
top             Y
freq       645188
Name: appointment_yosi_noshow1, dtype: object

In [35]:
location['distance'].describe()

count    1.048575e+06
mean     1.639536e+01
std      5.985731e+00
min      7.431175e-01
25%      1.201830e+01
50%      1.765740e+01
75%      2.152046e+01
max      2.499829e+01
Name: distance, dtype: float64

In [36]:
location['distance'] = pd.to_numeric(location['distance'], errors='coerce')
location['location_SON'] = location['distance'].apply(lambda x: 'too far_no show ' if x > 5 else 'closer_show up')
location.drop('distance', axis=1, inplace=True)
location['location_SON'].value_counts()


too far_no show     1004736
closer_show up        43839
Name: location_SON, dtype: int64

In [37]:
X = location.drop(columns=['location_SON'])

In [38]:
y = location["location_SON"]

In [39]:
X = location.drop(columns=['location_SON'])

In [40]:
X

,Unnamed: 0,id,practice_id_,appointment_date_time,patient_zipcode,appointment_yosi_noshow1,practice_id_update,practice_name,practice_zipcode
0,0,1165206,506,4/14/2021 12:14,19127.0,N,506,Concorde Medical Group PLLC,10016
1,1,1165206,506,4/14/2021 12:14,19127.0,N,506,Cornell Physicians - ENT,10021
2,2,1165206,506,4/14/2021 12:14,19127.0,N,506,Weill Cornell Brain And Spine Center,11355
3,3,1165206,506,4/14/2021 12:14,19127.0,N,506,Medical Associates Of Marlboro,8857
4,4,1165206,506,4/14/2021 12:14,19127.0,N,506,vybe urgent care Roxborough,19128
...,...,...,...,...,...,...,...,...,...
1048570,1048570,224565,501,9/30/2019 8:20,77084.0,Y,501,Shift Integrative Medicine,10003
1048571,1048571,224565,501,9/30/2019 8:20,77084.0,Y,501,Manhattan Pediatrics,10021
1048572,1048572,224565,501,9/30/2019 8:20,77084.0,Y,501,"Medical Imaging of Manhattan, LLC",10022
1048573,1048573,224565,501,9/30/2019 8:20,77084.0,Y,501,NYC Back Chiropractic,10128


In [41]:
y

0          too far_no show 
1          too far_no show 
2          too far_no show 
3          too far_no show 
4          too far_no show 
                 ...       
1048570    too far_no show 
1048571    too far_no show 
1048572    too far_no show 
1048573    too far_no show 
1048574    too far_no show 
Name: location_SON, Length: 1048575, dtype: object

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)

In [43]:
X_test

,Unnamed: 0,id,practice_id_,appointment_date_time,patient_zipcode,appointment_yosi_noshow1,practice_id_update,practice_name,practice_zipcode
5860,5860,732356,801,6/19/2020 7:30,73129.0,Y,801,"Francesco Rotatori, MD",10310
315344,315344,742769,801,7/22/2020 10:15,73401.0,Y,801,Gioffre Chiropractic Wellness Center,10075
534259,534259,278040,601,7/29/2021 15:00,80229.0,Y,601,Jay Ziegenfuss,18901
733669,733669,767251,801,10/20/2020 10:45,73763.0,N,801,Jason Chen,19312
438666,438666,291042,601,12/7/2021 16:15,80030.0,N,601,Jeffrey Case,7728
...,...,...,...,...,...,...,...,...,...
20372,20372,727532,801,6/1/2020 9:30,73049.0,Y,801,Karen Sobers-brown,8012
53509,53509,257621,601,1/18/2021 10:00,80229.0,Y,601,Rajendra Gupta,8618
280785,280785,219460,501,6/28/2019 15:00,77449.0,Y,501,Sissy John,19462
681843,681843,961669,503,6/25/2021 15:00,8081.0,Y,503,Cornell Physicians - ENT,10024


In [44]:
automl = AutoML(results_path="location_binary_los", mode="Explain")

In [45]:
automl.fit(X_train, y_train)  

Linear algorithm was disabled.
AutoML directory: location_binary_los
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline: trained.
2_DecisionTree: trained.
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.000441 trained in 552.77 seconds
4_Default_NeuralNetwork logloss 0.033265 trained in 1010.78 seconds
5_Default_RandomForest logloss 0.151081 trained in 183.6 seconds
* Step ensemble will try to check up to 1 model
There was an error during Ensemble training.
Please check location_binary_los/errors.md for details.
AutoML fit time: 1929.44 seconds
AutoML best model: 3_Default_Xgboost


AutoML(results_path='location_binary_los')

In [46]:
pred = automl.predict(X_test)
pred

array(['too far_no show ', 'too far_no show ', 'too far_no show ', ...,
       'too far_no show ', 'too far_no show ', 'too far_no show '],
      dtype=object)

In [47]:
automl.report()

In [48]:
# load in the data model 

automl_location_los = AutoML(results_path="location_binary_los")

In [49]:
# create a new dataset that follows the same data structure as the training set
X_withlos = location.sample(25)
X_withoutlos = X_withlos.drop(columns=['location_SON'])

In [50]:
X_withlos

,Unnamed: 0,id,practice_id_,appointment_date_time,patient_zipcode,appointment_yosi_noshow1,practice_id_update,practice_name,practice_zipcode,location_SON
808895,808895,646967,801,7/19/2019 10:00,73717.0,Y,801,Manhattan Dentistry,10111,too far_no show
1004961,1004961,84666,901,10/3/2020 10:40,33462.0,Y,901,Virginia Roth,17110,too far_no show
640593,640593,150285,401,8/24/2017 12:00,85202.0,Y,401,John Valeri,7920,too far_no show
64615,64615,775686,801,11/23/2020 10:00,73644.0,Y,801,Thomson Orthodontics,84341,too far_no show
976130,976130,235231,501,3/24/2020 9:00,77449.0,N,501,Qingping Wang,19446,too far_no show
874963,874963,274143,601,6/25/2021 8:00,80022.0,Y,601,Steven Ventrudo,10983,too far_no show
522919,522919,285030,601,10/8/2021 9:20,80221.0,Y,601,Brian Wolfman,8724,too far_no show
257445,257445,250324,601,10/30/2020 15:00,80030.0,N,601,Tommy Ng,8050,too far_no show
586356,586356,689426,801,1/8/2020 9:15,73112.0,N,801,Rashid Janjua,8619,too far_no show
864708,864708,408005,201,1/16/2020 13:50,85255.0,Y,201,Test Practice For Yosi,10016,too far_no show


In [51]:
X_withoutlos

,Unnamed: 0,id,practice_id_,appointment_date_time,patient_zipcode,appointment_yosi_noshow1,practice_id_update,practice_name,practice_zipcode
808895,808895,646967,801,7/19/2019 10:00,73717.0,Y,801,Manhattan Dentistry,10111
1004961,1004961,84666,901,10/3/2020 10:40,33462.0,Y,901,Virginia Roth,17110
640593,640593,150285,401,8/24/2017 12:00,85202.0,Y,401,John Valeri,7920
64615,64615,775686,801,11/23/2020 10:00,73644.0,Y,801,Thomson Orthodontics,84341
976130,976130,235231,501,3/24/2020 9:00,77449.0,N,501,Qingping Wang,19446
874963,874963,274143,601,6/25/2021 8:00,80022.0,Y,601,Steven Ventrudo,10983
522919,522919,285030,601,10/8/2021 9:20,80221.0,Y,601,Brian Wolfman,8724
257445,257445,250324,601,10/30/2020 15:00,80030.0,N,601,Tommy Ng,8050
586356,586356,689426,801,1/8/2020 9:15,73112.0,N,801,Rashid Janjua,8619
864708,864708,408005,201,1/16/2020 13:50,85255.0,Y,201,Test Practice For Yosi,10016


In [52]:
predict = automl.predict(X_withoutlos)
predict

array(['too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show ', 'too far_no show ', 'too far_no show ',
       'too far_no show '], dtype=object)

In [53]:
# actual values from X_withlos
values_actual = X_withlos['location_SON'].values.tolist()
values_predicted = predict.tolist()
output = pd.DataFrame({'actual': values_actual, 'predicted': values_predicted})
output

,actual,predicted
0,too far_no show,too far_no show
1,too far_no show,too far_no show
2,too far_no show,too far_no show
3,too far_no show,too far_no show
4,too far_no show,too far_no show
5,too far_no show,too far_no show
6,too far_no show,too far_no show
7,too far_no show,too far_no show
8,too far_no show,too far_no show
9,too far_no show,too far_no show


In [54]:
# get current working directory
import os
os.getcwd()

'/content'

In [56]:
folders = os.listdir()
foldersML = [x for x in folders if x.startswith('location')]
print(foldersML)

['location_binary_los']


In [57]:
!zip -r /content/location_binary_los.zip.zip /content/location_binary_los

  adding: content/location_binary_los/ (stored 0%)
  adding: content/location_binary_los/Ensemble/ (stored 0%)
  adding: content/location_binary_los/params.json (deflated 62%)
  adding: content/location_binary_los/ldb_performance.png (deflated 26%)
  adding: content/location_binary_los/errors.md (deflated 65%)
  adding: content/location_binary_los/data_info.json (deflated 74%)
  adding: content/location_binary_los/README.html (deflated 35%)
  adding: content/location_binary_los/README.md (deflated 66%)
  adding: content/location_binary_los/progress.json (deflated 89%)
  adding: content/location_binary_los/features_heatmap.png (deflated 17%)
  adding: content/location_binary_los/5_Default_RandomForest/ (stored 0%)
  adding: content/location_binary_los/5_Default_RandomForest/cumulative_gains_curve.png (deflated 6%)
  adding: content/location_binary_los/5_Default_RandomForest/learner_fold_0_shap_dependence.png (deflated 7%)
  adding: content/location_binary_los/5_Default_RandomForest/stat